In [1]:
import itertools

In [2]:
with open('day7.input') as fp:
    puzzle_prog = [int(s) for s in fp.read().split(',')]
puzzle_prog[:5]

[3, 8, 1001, 8, 10]

In [3]:
test1 = {'signal': 43210, 'phases': [4,3,2,1,0],
         'prog': [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]}
test2 = {'signal': 54321, 'phases': [0,1,2,3,4],
         'prog': [3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0]}
test3 = {'signal': 65210, 'phases': [1,0,4,3,2],
         'prog': [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]}

## Part 1 ##

In [4]:
mp = {1: 3, 2: 3, 3: 1, 4: 1,
      5: 2, 6: 2, 7: 3, 8: 3,
      99: 0} # mode : num parameters

def parse_opcode(opcode):
    """Given a numerica opcode value, return the code, and the parameter mode for each parameter."""
    s = str(opcode)
    if len(s) <= 2:
        # no parameter modes, so all parameters are 0
        return opcode, [0 for i in range(mp[opcode])]
    modestr, code = reversed(s[:-2]), int(s[-2:])
    modes = [int(c) for c in modestr]
    for i in range(len(modes), mp[code]):
        modes.append(0)
    return code, modes

In [5]:
def intcode_run(inp, d):
    outputs = []
    pos = 0
    while True:
        code, modes = parse_opcode(d[pos])
        if code == 99:
            break
        elif code == 1:
            i1, i2, o = d[pos+1:pos+4]
            if modes[0] == 0:
                i1 = d[i1]
            if modes[1] == 0:
                i2 = d[i2]
            if modes[2] != 0:
                raise ValueError(f'output mode must be 0, but is {modes[2]}')
            d[o] = i1 + i2
            pos += 4
        elif code == 2:
            i1, i2, o = d[pos+1:pos+4]
            if modes[0] == 0:
                i1 = d[i1]
            if modes[1] == 0:
                i2 = d[i2]
            if modes[2] != 0:
                raise ValueError(f'output mode must be 0, but is {modes[2]}')
            d[o] = i1 * i2
            pos += 4
        elif code == 3:
            if modes[0] != 0:
                raise ValueError(f'output mode must be 0, but is {modes[0]}')
            d[d[pos+1]] = inp.pop()
            pos += 2
        elif code == 4:
            i = d[pos+1]
            if modes[0] == 0:
                i = d[i]
            outputs.append(i)
            pos += 2
        elif code == 5:
            i1, i2 = d[pos+1], d[pos+2]
            if modes[0] == 0:
                i1 = d[i1]
            if modes[1] == 0:
                i2 = d[i2]
            if bool(i1):
                pos = i2
            else:
                pos += 3
        elif code == 6:
            i1, i2 = d[pos+1], d[pos+2]
            if modes[0] == 0:
                i1 = d[i1]
            if modes[1] == 0:
                i2 = d[i2]
            if not bool(i1):
                pos = i2
            else:
                pos += 3
        elif code == 7:
            i1, i2, o = d[pos+1:pos+4]
            if modes[0] == 0:
                i1 = d[i1]
            if modes[1] == 0:
                i2 = d[i2]
            d[o] = int(i1 < i2)
            pos += 4
        elif code == 8:
            i1, i2, o = d[pos+1:pos+4]
            if modes[0] == 0:
                i1 = d[i1]
            if modes[1] == 0:
                i2 = d[i2]
            d[o] = int(i1 == i2)
            pos += 4
        else:
            raise ValueError(f'Invalid opcode: {code}')
    return outputs

In [6]:
def amp_pipeline(phases, prog):
    out = 0
    for i in range(5):
        inp_vec = [out, phases[i]]
        out = intcode_run(inp_vec, prog.copy())[0]
    return out

In [7]:
for tst in (test1, test2, test3):
    print(tst['signal'] == amp_pipeline(tst['phases'], tst['prog']))

True
True
True


In [8]:
amp_pipeline(test1['phases'], test1['prog'])

43210

In [9]:
max(amp_pipeline(phases, puzzle_prog) for phases in itertools.permutations([0, 1, 2, 3, 4]))

366376

## Part 2 ##

In [10]:
intcode_run([112, 5], puzzle_prog.copy())

IndexError: pop from empty list

Need to define I/O streams for each pair of amps,and also each amp need to intialize and run until halted, but blocking while waiting for input from the previous amp.